![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported.

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

# Procedure:

In [ ]:
import pandas as pd
import numpy as np

# Start coding here...
df = pd.read_csv("bank_marketing.csv")

client = df[["client_id", "age", "job", "marital", "education", "credit_default", "mortgage"]]

# client_id is ok
# age column is ok

# job:
client["job"] = client["job"].str.replace(".", "_")
# marital is ok

# education:
client["education"] = client["education"].str.replace(".", "_")
client["education"] = client["education"].fillna(np.NaN)

# credit_default:
client["credit_default"] = client["credit_default"].apply(lambda x: 1 if x=="yes" else 0)
client["credit_default"] = client["credit_default"].astype("bool")

# mortgage
client["mortgage"] = client["mortgage"].apply(lambda x: 1 if x=="yes" else 0)
client["mortgage"] = client["mortgage"].astype("bool")

client.to_csv("client.csv")

In [ ]:
campaign = df[["client_id", "number_contacts", "contact_duration", "previous_campaign_contacts", "previous_outcome", "campaign_outcome", "month", "day"]]

# client_id is ok
# number_contacts is ok
# contact duration is ok
# previous_campaign_contacts is ok

# previous_outcome
campaign["previous_outcome"] = campaign["previous_outcome"].apply(lambda x: 1 if x== "success" else 0)
campaign["previous_outcome"] = campaign["previous_outcome"].astype("bool")

# campaign_outcome
campaign["campaign_outcome"] = campaign["campaign_outcome"].apply(lambda x: 1 if x== "yes" else 0)
campaign["campaign_outcome"] = campaign["campaign_outcome"].astype("bool")

# importing datetime module for now()
#from datetime import datetime
campaign["year"] = "2022"
mapping = {
    "mar" : 3,
    "apr" : 4,
    "may" : 5,
    "jun" : 6,
    "jul" : 7,
    "aug" : 8,
    "sep" : 9,
    "oct" : 10,
    "nov" : 11,
    "dic" : 12
}
campaign["month"] = campaign["month"].map(mapping)
campaign["last_contact_date"] = pd.to_datetime(campaign[['year', 'month', 'day']]).dt.strftime('%Y-%m-%d')
campaign["last_contact_date"] = pd.to_datetime(campaign["last_contact_date"])

campaign.drop(["year", "month", "day"], axis= 1, inplace= True)

campaign.to_csv("campaign.csv")

campaign.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   client_id                   41188 non-null  int64         
 1   number_contacts             41188 non-null  int64         
 2   contact_duration            41188 non-null  int64         
 3   previous_campaign_contacts  41188 non-null  int64         
 4   previous_outcome            41188 non-null  bool          
 5   campaign_outcome            41188 non-null  bool          
 6   last_contact_date           41006 non-null  datetime64[ns]
dtypes: bool(2), datetime64[ns](1), int64(4)
memory usage: 1.6 MB


In [ ]:
economics = df[["client_id", "cons_price_idx", "euribor_three_months"]]

economics.to_csv("economics.csv")

In [ ]:
#df = pd.read_csv("bank_marketing.csv")

for col in list(client.columns):
    print(col)
    print("--------------")
    print(df[col].value_counts())

client_id
--------------
0        1
27512    1
27454    1
27455    1
27456    1
        ..
13730    1
13731    1
13732    1
13733    1
41187    1
Name: client_id, Length: 41188, dtype: int64
age
--------------
31    1947
32    1846
33    1833
36    1780
35    1759
      ... 
89       2
91       2
94       1
87       1
95       1
Name: age, Length: 78, dtype: int64
job
--------------
admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
unknown            330
Name: job, dtype: int64
marital
--------------
married     24928
single      11568
divorced     4612
unknown        80
Name: marital, dtype: int64
education
--------------
university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y             